In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw

sheets = pd.read_excel('../data/lib/selleckchem-plate.xlsx', sheet_name=None)
sheets.keys()

dict_keys(['General Information', 'L1300-FDA-978cpds', 'Protein', "Laura's output", 'JAMES WORK 2', '20180410 JAMESOUTPUT', '20180419-classifier data', '20180510 - super python'])

In [2]:
info = sheets['General Information']
cpds = sheets['L1300-FDA-978cpds']

,FDA Approved Drug Screening Library (96-well),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,For more information on Selleck library servic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Product Details:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Formulation,NaN,A collection of 978 FDA approved drugs supplie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
import os
import os.path as osp
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
 
if not osp.exists('img'):
    os.mkdir('img')

mols = [Chem.MolFromSmiles(i) for i in layout.SMILES]
df = layout.copy()
df['mols'] = mols
df['img_path'] = df['Item Name'].apply(lambda name : \
               osp.join('img',name.lower().\
                        replace(' ','-').\
                        replace('(','').\
                        replace(')','') + '.png'))

for mol, name, path in tqdm(zip(df['mols'], df['Item Name'], df['img_path']), 
              total=len(df)):
    im = Draw.MolToImage(mol,fitImage=True,size=(300,300))
    im.save(path)

  0%|          | 0/978 [00:00<?, ?it/s]

In [25]:
from IPython.display import Image, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import *

path_to_image_html = \
    lambda path : f'<img src="{path}" style=max-height:1024px;"/>'

def show_im():
    CSS = """
    .output {
        flex-direction: row;
    }
    """
    HTML('<style>{}</style>'.format(CSS))

def image_viewer(cpd_name):
    paths = df.loc[df['Item Name'] == cpd_name, 'img_path']
    for i in paths:
        im = plt.imread(i)
        plt.figure(figsize=(8,8))
        plt.imshow(im)
        plt.show()
    
display_widget = widgets.Dropdown(options=df['Item Name'])
tab_nest = widgets.Tab()
tab_nest.set_title(0, 'Compounds')
f1 = interactive(image_viewer, 
                 cpd_name=display_widget);
tab_nest.children = [VBox(children = f1.children)]
display(tab_nest)

In [34]:
from rdkit.Chem import AllChem
import numpy as np
fps = np.array([AllChem.RDKFingerprint(i) for i in mols])
fps

array([[1, 1, 0, ..., 1, 0, 1],
       [1, 0, 1, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [1, 0, 1, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 0, 1]])

## Entropy

$$
H = - \sum{p(x_i) \log_2 p(x_i)}
$$

In [87]:
import random
from scipy.special import entr

def select(fps, n):
    p = fps / fps.sum(axis=0)
    #h = entr(p).sum(axis=0)
    pop_idx = random.choices(range(fps.shape[0]), k=n)
    h = lambda idx : entr(p[pop_idx]).sum()
    the_rest = lambda idx : [i for i in range(fps.shape[0]) \
                                 if i not in idx]
    H = h(pop_idx)
    for _ in range(100):
        trial_idx = pop_idx.copy()
        sample = random.choice(the_rest(trial_idx))
        trial_idx.remove(random.choice(trial_idx))
        trial_idx.append(sample)
        print(h(trial_idx))
        if h(trial_idx) > H:
            pop_idx = trial_idx
            H = h(trial_idx)
            print(H)
        
    
select(fps, 96)

1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903116907
1221.7207903

In [80]:
l=list(range(10))
print(l)
print(random.choice(l))
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [79]:
random.choice

Signature: random.choice(seq)
Docstring: Choose a random element from a non-empty sequence.
File:      ~/src/miniconda/miniconda3/envs/rdk/lib/python3.7/random.py
Type:      method
